<a href="https://colab.research.google.com/github/neenumohan/LSTM/blob/main/LSTM5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LSTM-0

In [ ]:
#working
from keras.models import Sequential
from keras.layers import Dense, LSTM
import numpy as np
# create sequence
seq1= [i/float(10) for i in range(0,10)] # [0, 0.1, 0.2 ..0.9]
seq2 = [i/float(10) for i in range(1,11)] # [0.1, 0.2 ..0.9, 1.0]
# Reformat to X and Y
X=np.array(seq1); Y=np.array(seq2)
X=X.reshape(10,1);Y=Y.reshape(10,1)

model = Sequential()
model.add(LSTM(50, input_shape=(10,1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X, Y, epochs=10000, batch_size=len(X), verbose=0)
loss = model.evaluate(X, Y, verbose=0)
print(model.predict(X, verbose=0))
print(loss)

[[0.10017182]
 [0.19978593]
 [0.29985672]
 [0.4000486 ]
 [0.5001639 ]
 [0.60013354]
 [0.69999367]
 [0.79985154]
 [0.89985174]
 [1.0001432 ]]


LSTM-5  Text Classification- Direct Data

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd

In [ ]:
df = pd.read_csv('sentiments.csv')
df.columns = ["label","text"]
x = df['text'].values
y = df['label'].values


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

In [ ]:
tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(x)
xtrain= tokenizer.texts_to_sequences(x_train)
xtest= tokenizer.texts_to_sequences(x_test)

In [ ]:
xtrain=pad_sequences(xtrain,padding='post', maxlen=10)
xtest=pad_sequences(xtest,padding='post', maxlen=10)

In [ ]:
print(x_train[3])

The service is a nightmare


In [ ]:
print(xtrain[3])

[86 87  4  1 88  0  0  0  0  0]


In [ ]:
embedding_dim=50
model=Sequential()
model.add(layers.Embedding(input_dim=len(tokenizer.word_index)+1,
      output_dim=embedding_dim,
      input_length=10))
model.add(layers.LSTM(units=50,return_sequences=True))
model.add(layers.LSTM(units=10))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8))
model.add(layers.Dense(1))
model.compile(optimizer="adam", loss="binary_crossentropy",
     metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 50)            4450      
                                                                 
 lstm (LSTM)                 (None, 10, 50)            20200     
                                                                 
 lstm_1 (LSTM)               (None, 10)                2440      
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense (Dense)               (None, 8)                 88        
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 27,187
Trainable params: 27,187
Non-trai

In [ ]:
model.fit(xtrain,y_train, epochs=20, batch_size=16, verbose=False)

In [ ]:
loss, acc = model.evaluate(xtrain, y_train, verbose=False)
print("Training Accuracy: ", round(acc,2))

Training Accuracy:  1.0


In [ ]:
loss, acc = model.evaluate(xtest, y_test, verbose=False)
print("Test Accuracy: ", round(acc,2))

Test Accuracy:  1.0


In [ ]:
ypred=model.predict(xtest)

1/1 [==============================] - 1s 851ms/step


In [ ]:
ypred[ypred>0.5]=1
ypred[ypred<=0.5]=0
print(confusion_matrix(y_test, ypred))

[[2 0]
 [0 4]]


In [ ]:
result=zip(x_test, y_test, ypred)
for i in result:
 print(i)

('I am excited a lot ', 1, array([1.], dtype=float32))
('exciting, liked. ', 1, array([1.], dtype=float32))
('terrible! I did not expect. ', 0, array([0.], dtype=float32))
('What a nice restaurant.', 1, array([1.], dtype=float32))
('not recommend, not satisfied ', 0, array([0.], dtype=float32))
('What a nice show.', 1, array([1.], dtype=float32))


LSTM-6 Text Classification with Pandas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Model
from keras.models import Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.utils import pad_sequences
%matplotlib inline

In [ ]:
df = pd.read_csv('spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [ ]:
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [ ]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

In [ ]:
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(X)
xtrain= tokenizer.texts_to_sequences(X_train)
xtest= tokenizer.texts_to_sequences(X_test)

In [ ]:
model=Sequential()
model.add(keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1,
      output_dim=50,input_length=150))
model.add(keras.layers.LSTM(units=64,return_sequences=True))
model.add(keras.layers.Dense(256,name='FC1',activation="sigmoid"))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(1, name='out_layer', activation="sigmoid"))
model.compile(optimizer="adam", loss="binary_crossentropy",
     metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 150, 50)           446050    
                                                                 
 lstm (LSTM)                 (None, 150, 64)           29440     
                                                                 
 FC1 (Dense)                 (None, 150, 256)          16640     
                                                                 
 dropout (Dropout)           (None, 150, 256)          0         
                                                                 
 out_layer (Dense)           (None, 150, 1)            257       
                                                                 
Total params: 492,387
Trainable params: 492,387
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 150, 50)           446050    
                                                                 
 lstm (LSTM)                 (None, 150, 64)           29440     
                                                                 
 FC1 (Dense)                 (None, 150, 256)          16640     
                                                                 
 dropout (Dropout)           (None, 150, 256)          0         
                                                                 
 out_layer (Dense)           (None, 150, 1)            257       
                                                                 
Total params: 492,387
Trainable params: 492,387
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
30/30 [==============================] - 18s 508ms/step - loss: 0.4175 - accuracy: 0.8666 - val_loss: 0.3890 - val_accuracy: 0.8681
Epoch 2/10
30/30 [==============================] - 15s 489ms/step - loss: 0.4085 - accuracy: 0.8681 - val_loss: 0.3756 - val_accuracy: 0.8728
Epoch 3/10
30/30 [==============================] - 15s 497ms/step - loss: 0.3904 - accuracy: 0.8760 - val_loss: 0.3797 - val_accuracy: 0.8771


In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
accr = model.evaluate(test_sequences_matrix,Y_test)

27/27 [==============================] - 1s 35ms/step - loss: 0.4123 - accuracy: 0.8635


In [ ]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.412
  Accuracy: 0.864
